## Scratchpad

In [6]:
import env
import pandas as pd
import numpy as np
import os

In [11]:
def get_connection(db, user=env.username, host=env.hostname, password=env.password):
    connection_info = f'mysql+pymysql://{user}:{password}@{host}/{db}'
    return connection_info

In [18]:
def get_zillow_data():
    '''
    Function to retrieve the 2017 Zillow Property Data Set from CODEUP's mySQL Server
    '''
    if os.path.isfile('2017_zillow_hot_month_properties.csv'):
        df = pd.read_csv('2017_zillow_hot_month_properties.csv', index_col=0)  # If csv file exists read in data from csv file.
    else:
        sql = '''
                SELECT bedroomcnt, bathroomcnt, 
                    calculatedfinishedsquarefeet, 
                    taxvaluedollarcnt, yearbuilt, 
                    taxamount, fips, regionidzip 
                FROM properties_2017
                JOIN predictions_2017 USING(id)
                WHERE propertylandusetypeid = 261
                    AND transactiondate BETWEEN '2017-05-01' AND '2017-09-01';
                '''   # SQL query
                                                    
        db = 'zillow'                                   # Database name
        df = pd.read_sql(sql, get_connection(db))       # Pandas DataFrame
        df.to_csv('2017_zillow_hot_month_properties.csv')         # Cache Data
    return df

In [19]:
def get_zipcode_data():
    '''
    Function to retrieve the 2016 Zillow average zipcode Data from CODEUP's mySQL Server
    '''
    if os.path.isfile('2016_zillow_zipcodes.csv'):
        df = pd.read_csv('2016_zillow_zipcodes.csv', index_col=0)  # If csv file exists read in data from csv file.
    else:
        sql = '''
                SELECT COUNT(regionidzip) AS zipcode_count, 
                    regionidzip AS zipcode, 
                    ROUND(AVG(taxvaluedollarcnt),0) AS zipcode_avg_price
                FROM properties_2016
                WHERE propertylandusetypeid = 261
                GROUP BY regionidzip
                ORDER BY AVG(taxvaluedollarcnt) DESC;'''   # SQL query
                                                    
        db = 'zillow'                                   # Database name
        df = pd.read_sql(sql, get_connection(db))       # Pandas DataFrame
        df.to_csv('2016_zillow_zipcodes.csv')         # Cache Data
    return df

In [20]:
zillow = get_zillow_data()
zillow.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,regionidzip
0,3.0,2.0,1772.0,418763.0,1978.0,4464.42,6111.0,97089.0
1,3.0,2.0,1819.0,478430.0,1981.0,5515.94,6111.0,97118.0
2,2.0,2.0,1503.0,171687.0,1933.0,2287.55,6037.0,96414.0
3,4.0,2.0,1604.0,498347.0,1950.0,6089.82,6037.0,96415.0
4,3.0,3.0,2384.0,549917.0,1937.0,6679.55,6037.0,96452.0


In [21]:
zips = get_zipcode_data()
zips.head()

,zipcode_count,zipcode,zipcode_avg_price
0,7103,96086.0,2847596.0
1,2856,96975.0,2768183.0
2,3102,96058.0,2607519.0
3,5625,96116.0,2186823.0
4,7490,96030.0,1822090.0


In [27]:
zips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   zipcode_count      399 non-null    int64  
 1   zipcode            398 non-null    float64
 2   zipcode_avg_price  399 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 9.5 KB


In [39]:
zips.isnull().sum()

zipcode_count        0
zipcode              1
zipcode_avg_price    0
dtype: int64

In [40]:
zips = zips.replace(r'^\s*$', np.nan, regex=True)

In [41]:
zips.isnull().sum()

zipcode_count        0
zipcode              1
zipcode_avg_price    0
dtype: int64

In [42]:
zips = zips.dropna()
zips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398 entries, 0 to 398
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   zipcode_count      398 non-null    int64  
 1   zipcode            398 non-null    float64
 2   zipcode_avg_price  398 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 12.4 KB


In [43]:
zips = zips.astype('int64')
zips.head()

,zipcode_count,zipcode,zipcode_avg_price
0,7103,96086,2847596
1,2856,96975,2768183
2,3102,96058,2607519
3,5625,96116,2186823
4,7490,96030,1822090
